# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Path to csv file
file_one = "output_data/cities.csv"

# Read csv file
cities_df = pd.read_csv(file_one)

# Display head of csv file
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,yaan,7.3833,8.5667,69.87,90,99,3.76,NG,1627453711
1,paamiut,61.9940,-49.6678,44.46,82,83,9.13,GL,1627453712
2,iqaluit,63.7506,-68.5145,42.53,81,75,5.01,CA,1627453712
3,mataura,-46.1927,168.8643,48.47,93,89,2.55,NZ,1627453712
4,vardo,70.3705,31.1107,50.20,93,75,10.36,NO,1627453713


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity Rate to float and store
humidity_rate = cities_df["Humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Ideal weather conditions
new_cities_df = cities_df.loc[(cities_df["Humidity"] < 80) & (cities_df["Humidity"] > 70) & (cities_df["Cloudiness"] < 20) & (cities_df["Wind Speed"] < 10), :]
new_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,luderitz,-26.6481,15.1594,53.38,72,0,9.71,NaN,1627453719
88,east london,-33.0153,27.9116,62.11,79,13,8.21,ZA,1627453748
128,cabo san lucas,22.8909,-109.9124,85.60,74,16,9.15,MX,1627453766
190,terrassa,41.5667,2.0167,75.33,71,0,1.99,ES,1627453796
202,anori,-3.7728,-61.6442,71.89,73,6,0.16,BR,1627453804
227,san patricio,28.0170,-97.5169,84.09,77,0,5.95,US,1627453817
240,flin flon,54.7682,-101.8650,59.00,76,6,4.94,CA,1627453822
244,tautira,-17.7333,-149.1500,77.02,71,14,7.87,PF,1627453825
376,lorengau,-2.0226,147.2712,83.16,77,1,5.64,PG,1627453890
413,barcelos,41.5388,-8.6151,66.92,74,6,4.23,PT,1627453908


In [27]:
# Drop any rows with null values
new_cities_df = new_cities_df.dropna()
# None had null values

# Display data
new_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,east london,-33.0153,27.9116,62.11,79,13,8.21,ZA,1627453748
128,cabo san lucas,22.8909,-109.9124,85.60,74,16,9.15,MX,1627453766
190,terrassa,41.5667,2.0167,75.33,71,0,1.99,ES,1627453796
202,anori,-3.7728,-61.6442,71.89,73,6,0.16,BR,1627453804
227,san patricio,28.0170,-97.5169,84.09,77,0,5.95,US,1627453817
240,flin flon,54.7682,-101.8650,59.00,76,6,4.94,CA,1627453822
244,tautira,-17.7333,-149.1500,77.02,71,14,7.87,PF,1627453825
376,lorengau,-2.0226,147.2712,83.16,77,1,5.64,PG,1627453890
413,barcelos,41.5388,-8.6151,66.92,74,6,4.23,PT,1627453908
434,loreto,22.2667,-101.9667,61.84,71,9,6.91,MX,1627453918


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Create new dataframe with City, Country, Lat and Lng
hotel_df = new_cities_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Lat,Lng
88,east london,ZA,-33.0153,27.9116
128,cabo san lucas,MX,22.8909,-109.9124
190,terrassa,ES,41.5667,2.0167
202,anori,BR,-3.7728,-61.6442
227,san patricio,US,28.0170,-97.5169
240,flin flon,CA,54.7682,-101.8650
244,tautira,PF,-17.7333,-149.1500
376,lorengau,PG,-2.0226,147.2712
413,barcelos,PT,41.5388,-8.6151
434,loreto,MX,22.2667,-101.9667


In [29]:
# Add Hotel Name column
hotel_df["Hotel Name"]= ""

# Display hotel_df dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
88,east london,ZA,-33.0153,27.9116,
128,cabo san lucas,MX,22.8909,-109.9124,
190,terrassa,ES,41.5667,2.0167,
202,anori,BR,-3.7728,-61.6442,
227,san patricio,US,28.0170,-97.5169,
240,flin flon,CA,54.7682,-101.8650,
244,tautira,PF,-17.7333,-149.1500,
376,lorengau,PG,-2.0226,147.2712,
413,barcelos,PT,41.5388,-8.6151,
434,loreto,MX,22.2667,-101.9667,


In [30]:
# Create for loop to locate nearest hotel in city within 5000m

for idx,row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    # Geocoordinates
    target_coordinates = f"{Lat},{Lng}"
    target_radius = 5000
    target_type = "lodging"

    # Set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    # Base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    # Use try except in case there's no located hotel within target radius
    try:
        hotel_df.loc[idx,"Hotel Name"]=(response.json()["results"][0]["name"])
    except:
        print("No hotel found here")

# Print data frame
hotel_df   

,City,Country,Lat,Lng,Hotel Name
88,east london,ZA,-33.0153,27.9116,Tu Casa
128,cabo san lucas,MX,22.8909,-109.9124,Hotel Tesoro Los Cabos
190,terrassa,ES,41.5667,2.0167,Hotel Don Candido
202,anori,BR,-3.7728,-61.6442,Novo Hotel 05 Irmaos
227,san patricio,US,28.0170,-97.5169,"Motel 6 Sinton, TX"
240,flin flon,CA,54.7682,-101.8650,Victoria Inn Flin Flon
244,tautira,PF,-17.7333,-149.1500,Pension Oaoa
376,lorengau,PG,-2.0226,147.2712,Lorengau Harbourside Hotel
413,barcelos,PT,41.5388,-8.6151,Hotel Do Terço
434,loreto,MX,22.2667,-101.9667,José de Jesús Velásquez mora


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Create marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))